# Predicción de abandono del servicio en Telecom X

## Preparación de los datos

In [1]:
import pandas as pd

In [2]:
path = 'datos/datos_tratados.csv'
df = pd.read_csv(path)
df.sample(5)

,CustomerID,Churn,Gender,SeniorCitizen,Partner,Dependents,Tenure,PhoneService,MultipleLines,InternetService,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,ChargesDaily,ChargesMonthly,ChargesTotal
1332,1982-FEBTD,No,Female,No,Yes,Yes,23,Yes,Yes,No,...,No,No,No,No,Two year,Yes,Credit card (automatic),0.85,25.60,514.75
6317,8944-AILEF,No,Male,No,Yes,No,15,Yes,No,No,...,No,No,No,No,Month-to-month,Yes,Credit card (automatic),0.65,19.45,340.85
650,0952-KMEEH,Yes,Male,No,No,No,13,Yes,Yes,Fiber optic,...,No,No,Yes,Yes,Month-to-month,No,Mailed check,3.27,98.15,1230.25
4059,5716-LIBJC,No,Female,No,No,Yes,13,Yes,Yes,Fiber optic,...,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),3.47,104.15,1299.10
6219,8791-GFXLZ,No,Male,No,No,No,12,Yes,No,DSL,...,No,No,No,No,Month-to-month,Yes,Bank transfer (automatic),1.64,49.05,593.05


In [3]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   CustomerID        7043 non-null   str    
 1   Churn             7043 non-null   str    
 2   Gender            7043 non-null   str    
 3   SeniorCitizen     7043 non-null   str    
 4   Partner           7043 non-null   str    
 5   Dependents        7043 non-null   str    
 6   Tenure            7043 non-null   int64  
 7   PhoneService      7043 non-null   str    
 8   MultipleLines     7043 non-null   str    
 9   InternetService   7043 non-null   str    
 10  OnlineSecurity    7043 non-null   str    
 11  OnlineBackup      7043 non-null   str    
 12  DeviceProtection  7043 non-null   str    
 13  TechSupport       7043 non-null   str    
 14  StreamingTV       7043 non-null   str    
 15  StreamingMovies   7043 non-null   str    
 16  Contract          7043 non-null   str    
 17  Paperl

In [4]:
df = df.drop(columns=['CustomerID'])

In [5]:
X = df.drop('Churn', axis=1)
y = df['Churn']

In [6]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder

In [7]:
categorical_features = df.columns.drop(['Churn', 'Tenure', 'ChargesDaily', 'ChargesMonthly', 'ChargesTotal'])

one_hot = make_column_transformer(
    (OneHotEncoder(drop='if_binary'), categorical_features),
    remainder='passthrough', sparse_threshold=0
)

In [8]:
X = one_hot.fit_transform(X)
X

array([[0.00000e+00, 0.00000e+00, 1.00000e+00, ..., 2.19000e+00,
        6.56000e+01, 5.93300e+02],
       [1.00000e+00, 0.00000e+00, 0.00000e+00, ..., 2.00000e+00,
        5.99000e+01, 5.42400e+02],
       [1.00000e+00, 0.00000e+00, 0.00000e+00, ..., 2.46000e+00,
        7.39000e+01, 2.80850e+02],
       ...,
       [1.00000e+00, 0.00000e+00, 0.00000e+00, ..., 1.68000e+00,
        5.03000e+01, 9.27500e+01],
       [1.00000e+00, 0.00000e+00, 1.00000e+00, ..., 2.26000e+00,
        6.78500e+01, 4.62765e+03],
       [1.00000e+00, 0.00000e+00, 1.00000e+00, ..., 1.97000e+00,
        5.90000e+01, 3.70760e+03]], shape=(7043, 27))

In [9]:
from sklearn.preprocessing import LabelEncoder

In [10]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)